<a href="https://colab.research.google.com/github/Nabeel-Rizwan/NLP/blob/main/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test_data (1).csv to test_data (1) (1).csv


In [ ]:

# Check if the uploaded object is a DataFrame
if isinstance(uploaded, pd.DataFrame):
    df = uploaded.copy()  # If already a DataFrame, make a copy
else:
    # If not a DataFrame, read it into a DataFrame
    try:
        df = pd.read_csv(next(iter(uploaded)))
    except Exception as e:
        print("Error:", e)
        print("Failed to read the uploaded file into a DataFrame.")
        df = pd.DataFrame()  # Empty DataFrame

# Check if DataFrame is not empty and proceed with operations
if not df.empty:
    # Selecting required columns and renaming them
    df = df[['Text', 'Score']]
    df['review'] = df['Text']
    df['rating'] = df['Score']
    df.drop(['Text', 'Score'], axis=1, inplace=True)
else:
    print("DataFrame is empty. Please make sure to upload a valid CSV file.")

In [ ]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.

style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#nltk
import nltk

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for part-of-speech tagging
from nltk import pos_tag

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

# BeautifulSoup libraray
from bs4 import BeautifulSoup

import re # regex

#model_selection
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

#evaluation
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.metrics import classification_report
from mlxtend.plotting import plot_confusion_matrix

#preprocessing scikit
#from sklearn.preprocessing import MinMaxScaler,StandardScaler,Imputer,LabelEncoder

#classifiaction.
from sklearn.linear_model import LogisticRegression


#stop-words
#stop_words=set(nltk.corpus.stopwords.words('english'))

#keras
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM
from keras.models import Model
from keras.preprocessing.text import text_to_word_sequence

#gensim w2v
#word2vec
from gensim.models import Word2Vec





In [ ]:
# remove duplicates/ for every duplicate we will keep only one row of that type.
df.drop_duplicates(subset=['rating','review'],keep='first',inplace=True)


In [ ]:
print(df.shape)
df.head()

(16745, 2)


,review,rating
0,I did not enjoy the film Eraser whatsoever. It...,0
1,Be very afraid of anyone who likes this film. ...,0
2,The 3rd and last big screen spin off from the ...,0
3,Barely three and a half years after just scrap...,1
4,I'm a big fan of the demonic puppets. Looking ...,0


In [ ]:
def mark_sentiment(rating):
    if(rating>=1):
        return 1
    else:
        return 0

df['sentiment']=df['rating'].apply(mark_sentiment)
df.drop(['rating'],axis=1,inplace=True)

# function to clean and pre-process the text.
def clean_reviews(review):

    # 1. Removing html tags
    review_text = BeautifulSoup(review,"lxml").get_text()

    # 2. Retaining only alphabets.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)

    # 3. Converting to lower case and splitting
    word_tokens= review_text.lower().split()

    # 4. Remove stopwords
    le=WordNetLemmatizer()
    stop_words= set(stopwords.words("english"))
    word_tokens= [le.lemmatize(w) for w in word_tokens if not w in stop_words]

    cleaned_review=" ".join(word_tokens)
    return cleaned_review

pos_df=df.loc[df.sentiment==1,:][:1000]
neg_df=df.loc[df.sentiment==0,:][:1000]

In [ ]:
#combining
df=pd.concat([pos_df,neg_df],ignore_index=True)
# shuffling rows
df = df.sample(frac=1).reset_index(drop=True)
print(df.shape)
df.head()

(2000, 2)


,review,sentiment
0,"Although Bullet In The Brain is, without quest...",1
1,"Like many a child born in the 1980's, I grew u...",1
2,"I watched ""9 souls"" in Athens' 12th Internatio...",1
3,I was unsure of this movie before renting and ...,1
4,Now I know why this movie can be bought for so...,0


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences=[]
sum=0
for review in df['review']:
    sents=tokenizer.tokenize(review.strip())
    sum+=len(sents)
    for sent in sents:
        cleaned_sent=clean_reviews(sent)
        sentences.append(cleaned_sent.split()) # can use word_tokenize also.
print(sum)
print(len(sentences))  # total no of sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


21613
21613


In [ ]:
# trying to print few sentences
for te in sentences[:5]:
    print(te,"\n")

['although', 'bullet', 'brain', 'without', 'question', 'superior', 'amongst', 'short', 'film', 'largely', 'seems', 'like', 'short', 'piece', 'writing', 'film'] 

['little', 'hard', 'feel', 'sorry', 'teacher', 'smart', 'as', 'remark', 'get', 'shot'] 

['bullet', 'enters', 'brain', 'begin', 'understand', 'little', 'bit', 'became', 'jaded', 'life', 'first', 'place'] 

['awful', 'amount', 'detail', 'packed', 'reasonably', 'short', 'film', 'make', 'feel', 'extended', 'little', 'bit', 'seems', 'like', 'almost', 'much', 'take', 'detail', 'come', 'flying', 'fast'] 

['slightly', 'relaxed', 'pace', 'le', 'po', 'faced', 'narrator', 'final', 'section', 'would', 'benefitted', 'film', 'little', 'bit'] 



In [ ]:
import gensim
w2v_model = gensim.models.Word2Vec(sentences=sentences, vector_size=100, window=10, min_count=1)

# w2v_model=gensim.models.Word2Vec(sentences=sentences,size=50,window=10,min_count=1)

In [ ]:
w2v_model.train(sentences,epochs=10,total_examples=len(sentences))
# embedding of a particular word.
w2v_model.wv.get_vector('death')

array([ 0.28815863,  0.80870295, -0.23667943, -0.5849514 ,  0.16243248,
       -1.208678  , -0.10651719,  0.6854493 , -0.666811  , -0.49967924,
       -0.7247034 , -0.616116  , -0.10810337,  0.13656405,  0.02024376,
       -0.00664509, -0.02476886, -1.7596862 ,  0.97069573, -0.665109  ,
        0.24493448,  0.89027864,  1.0525541 ,  0.48730132, -0.6151875 ,
       -0.00755355,  1.0988686 ,  0.01997111, -0.35379696, -0.3135022 ,
       -0.41094723, -0.07492305, -0.20520775,  0.54754734, -0.3153094 ,
        0.24196878, -0.83180016, -0.41540092, -0.45991474, -0.42486492,
        0.60244256, -0.73295414,  0.08838957,  0.19636327,  0.55142504,
        0.0556349 , -1.1799364 ,  0.7543188 , -0.86270547,  0.48571643,
       -0.04168688,  0.17024031, -0.13335234, -0.5212543 , -0.8155987 ,
        0.4142941 ,  0.07675225, -0.3799949 ,  0.05008825, -0.37081403,
        0.05381119,  0.24495946, -1.2318033 ,  0.70679677, -0.17963922,
        0.3779696 ,  0.57761407,  0.1541603 , -0.5277482 ,  1.24

In [ ]:
# total numberof extracted words.
# vocab=w2v_model.wv.vocab
# print("The total number of words are : ",len(vocab))

# total number of extracted words
vocab_keys = w2v_model.wv.index_to_key
print("The total number of words are:", len(vocab_keys))

The total number of words are: 22570


In [ ]:
# words most similar to a given word.
target_similar = w2v_model.wv.most_similar('fun')
print(target_similar)
# similaraity b/w two words
target_s=w2v_model.wv.similarity('fun','movie')
print(target_s)

[('scary', 0.945197582244873), ('enjoy', 0.938279390335083), ('eagerness', 0.926854133605957), ('entertaining', 0.9259776473045349), ('poking', 0.9247835874557495), ('sorry', 0.9234099388122559), ('worthwhile', 0.9230263829231262), ('laugh', 0.9191361665725708), ('disappointed', 0.9142984747886658), ('stuff', 0.9115363955497742)]
0.744691


In [ ]:
# cleaning reviews.
df['clean_review']=df['review'].apply(clean_reviews)


In [ ]:
# number of unique words = 589.

# now since we will have to pad we need to find the maximum lenght of any document.

maxi=-1
for i,rev in enumerate(df['clean_review']):
    tokens=rev.split()
    if(len(tokens)>maxi):
        maxi=len(tokens)
print(maxi)

589


In [ ]:
tok = Tokenizer()
tok.fit_on_texts(df['clean_review'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['clean_review'])

max_rev_len=maxi  # max lenght of a review
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim=100 # embedding dimension as choosen in word2vec constructor

# now padding to have a amximum length of 1565
pad_rev= pad_sequences(encd_rev, maxlen=max_rev_len, padding='post')
pad_rev.shape   # note that we had 100K reviews and we have padded each review to have  a lenght of 1565 words.

(2000, 589)

In [ ]:
# now creating the embedding matrix


embed_matrix = np.zeros(shape=(vocab_size, embed_dim))  # Adjust the dimension to match your Word2Vec embeddings
for word, i in tok.word_index.items():
    embed_vector = w2v_model.wv.get_vector(word)
    if embed_vector is not None:  # word is in the vocabulary learned by the Word2Vec model
        embed_matrix[i] = embed_vector

In [ ]:

from keras.layers import Embedding, Flatten, Dense, Dropout
from keras.initializers import Constant
import keras
from sklearn.model_selection import train_test_split

# Prepare train and val sets first
Y = keras.utils.to_categorical(df['sentiment'])  # One hot target as required by NN.
x_train, x_test, y_train, y_test = train_test_split(pad_rev, Y, test_size=0.20, random_state=42)

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=max_rev_len, embeddings_initializer=Constant(embed_matrix)))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(2, activation='sigmoid'))  # Sigmoid for binary classification.

# Compile the model
model.compile(optimizer=keras.optimizers.RMSprop(lr=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

# Specify batch size and epochs for training
epochs = 5
batch_size = 64

# Fitting the model
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test))

Epoch 1/5
25/25 [==============================] - 2s 48ms/step - loss: 0.7320 - accuracy: 0.5281 - val_loss: 0.6812 - val_accuracy: 0.5425
Epoch 2/5
25/25 [==============================] - 1s 60ms/step - loss: 0.6755 - accuracy: 0.5738 - val_loss: 0.6740 - val_accuracy: 0.6150
Epoch 3/5
25/25 [==============================] - 2s 63ms/step - loss: 0.6663 - accuracy: 0.6031 - val_loss: 0.6724 - val_accuracy: 0.5650
Epoch 4/5
25/25 [==============================] - 1s 47ms/step - loss: 0.6462 - accuracy: 0.6150 - val_loss: 0.6559 - val_accuracy: 0.6075
Epoch 5/5
25/25 [==============================] - 1s 39ms/step - loss: 0.6215 - accuracy: 0.6350 - val_loss: 0.6523 - val_accuracy: 0.6450
